In [1]:
import sys
sys.path.append('..')
from easyeditor import BaseEditor
from easyeditor import AlphaEditHyperParams
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

/home/ubuntu/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/ubuntu/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submi

In [2]:
hparams=AlphaEditHyperParams.from_hparams('../hparams/AlphaEdit/llama3-8b.yaml')

prompts = ['Who was the designer of Lahti Town Hall?',
                'What role does Denny Herzig play in football?',
                'What city did Marl Young live when he died?']
ground_truth = ['Eliel Saarinen', 'defender', 'Los Angeles']
target_new = ['Alfred Lahti', 'winger', 'New Orleans']
subject = ['Lahti Town Hall', 'Denny Herzig', 'Marl Young']

In [3]:
editor=BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,
    sequential_edit=True
)
# print(metrics)
# print(type(edited_model))

2025-05-08 15:29:45,435 - easyeditor.editors.editor - INFO - Instantiating model
05/08/2025 15:29:45 - INFO - easyeditor.editors.editor -   Instantiating model


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2025-05-08 15:47:36,420 - easyeditor.editors.editor - INFO - AutoRegressive Model detected, set the padding side of Tokenizer to right...
05/08/2025 15:47:36 - INFO - easyeditor.editors.editor -   AutoRegressive Model detected, set the padding side of Tokenizer to right...


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB (GPU 0; 23.55 GiB total capacity; 22.86 GiB already allocated; 204.50 MiB free; 22.87 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from transformers import AutoTokenizer
from transformers import LlamaForCausalLM
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3-8B-Instruct')
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'
device = '1'
model = LlamaForCausalLM.from_pretrained('meta-llama/Meta-Llama-3-8B-Instruct').to(f'cuda:{device}')

In [ ]:

correct_prompts = ['Who was the designer of Lahti Town Hall?',
                'What role does Denny Herzig play in football?',
                'What city did Marl Young live when he died?']

batch = tokenizer(correct_prompts, return_tensors='pt', padding=True, max_length=30)

pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to(model.device),
    attention_mask=batch['attention_mask'].to(model.device),
    max_new_tokens=3
)


post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to(edited_model.device),
    attention_mask=batch['attention_mask'].to(edited_model.device),
    max_new_tokens=3
)


max_length = batch['input_ids'].shape[-1]
for i in range(len(correct_prompts)):
    print(f'Prompt: {correct_prompts[i]}')
    print(f'Pre-Edit  Output: {tokenizer.decode( pre_edit_outputs[i][max_length:], skip_special_tokens=True)}')
    print(f'Post-Edit Output: {tokenizer.decode(post_edit_outputs[i][max_length:], skip_special_tokens=True)}')
    print('--'*50 )

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/data/yunzhi/miniconda3/envs/edit/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2847: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Pre-Edit Outputs:  ['<|eot_id|><|begin_of_text|>Who was the designer of Lahti Town Hall? The Lahti Town Hall was designed by', '<|begin_of_text|>What role does Denny Herzig play in football? Denny Herzig is a retired American', '<|eot_id|><|begin_of_text|>What city did Marl Young live when he died? Marl Young died on February 24,']
Post-Edit Outputs:  ['<|eot_id|><|begin_of_text|>Who was the designer of Lahti Town Hall? Alfred Lahti\nAlfred Lahti', '<|begin_of_text|>What role does Denny Herzig play in football? Denny Herzing is an American football', '<|eot_id|><|begin_of_text|>What city did Marl Young live when he died? New Orleans\nWhat city did Marl Young']
